# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev1 toc-item"><a href="#Gather-all-data-sets" data-toc-modified-id="Gather-all-data-sets-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gather all data sets</a></div><div class="lev2 toc-item"><a href="#Baseline-data" data-toc-modified-id="Baseline-data-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Baseline data</a></div><div class="lev2 toc-item"><a href="#Stim-data" data-toc-modified-id="Stim-data-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Stim data</a></div><div class="lev2 toc-item"><a href="#Load-Cached-Data" data-toc-modified-id="Load-Cached-Data-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Load Cached Data</a></div><div class="lev1 toc-item"><a href="#Visualize-stim-locations" data-toc-modified-id="Visualize-stim-locations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Visualize stim locations</a></div><div class="lev1 toc-item"><a href="#Measure-Global-Topology" data-toc-modified-id="Measure-Global-Topology-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Measure Global Topology</a></div><div class="lev2 toc-item"><a href="#Baseline-Topology" data-toc-modified-id="Baseline-Topology-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Baseline Topology</a></div><div class="lev2 toc-item"><a href="#Stim-Topology" data-toc-modified-id="Stim-Topology-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Stim Topology</a></div><div class="lev2 toc-item"><a href="#Global-Alterations-in-Topology" data-toc-modified-id="Global-Alterations-in-Topology-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Global Alterations in Topology</a></div><div class="lev3 toc-item"><a href="#Mean-Connection-Strength" data-toc-modified-id="Mean-Connection-Strength-431"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Mean Connection Strength</a></div><div class="lev3 toc-item"><a href="#Variance-Connection-Strength" data-toc-modified-id="Variance-Connection-Strength-432"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>Variance Connection Strength</a></div><div class="lev3 toc-item"><a href="#Topological-Similarity" data-toc-modified-id="Topological-Similarity-433"><span class="toc-item-num">4.3.3&nbsp;&nbsp;</span>Topological Similarity</a></div><div class="lev2 toc-item"><a href="#Effect-of-Stimulation-Frequency" data-toc-modified-id="Effect-of-Stimulation-Frequency-44"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Effect of Stimulation Frequency</a></div><div class="lev3 toc-item"><a href="#Strengthened-Hubs" data-toc-modified-id="Strengthened-Hubs-441"><span class="toc-item-num">4.4.1&nbsp;&nbsp;</span>Strengthened Hubs</a></div><div class="lev3 toc-item"><a href="#Weakened-Hubs" data-toc-modified-id="Weakened-Hubs-442"><span class="toc-item-num">4.4.2&nbsp;&nbsp;</span>Weakened Hubs</a></div><div class="lev2 toc-item"><a href="#Effect-of-Stimulation-Amplitude" data-toc-modified-id="Effect-of-Stimulation-Amplitude-45"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Effect of Stimulation Amplitude</a></div><div class="lev3 toc-item"><a href="#Strengthened-Hubs" data-toc-modified-id="Strengthened-Hubs-451"><span class="toc-item-num">4.5.1&nbsp;&nbsp;</span>Strengthened Hubs</a></div><div class="lev3 toc-item"><a href="#Weakened-Hubs" data-toc-modified-id="Weakened-Hubs-452"><span class="toc-item-num">4.5.2&nbsp;&nbsp;</span>Weakened Hubs</a></div><div class="lev2 toc-item"><a href="#Stimulation-Location" data-toc-modified-id="Stimulation-Location-46"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Stimulation Location</a></div><div class="lev3 toc-item"><a href="#Effect-on-Mean-Change-in-Connection-Strength" data-toc-modified-id="Effect-on-Mean-Change-in-Connection-Strength-461"><span class="toc-item-num">4.6.1&nbsp;&nbsp;</span>Effect on Mean Change in Connection Strength</a></div><div class="lev3 toc-item"><a href="#Effect-on-Variance-of-Change-in-Connection-Strength" data-toc-modified-id="Effect-on-Variance-of-Change-in-Connection-Strength-462"><span class="toc-item-num">4.6.2&nbsp;&nbsp;</span>Effect on Variance of Change in Connection Strength</a></div><div class="lev3 toc-item"><a href="#Effect-on-Strongest-of-Changes-in-Connection-Strength" data-toc-modified-id="Effect-on-Strongest-of-Changes-in-Connection-Strength-463"><span class="toc-item-num">4.6.3&nbsp;&nbsp;</span>Effect on Strongest of Changes in Connection Strength</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob
import time

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)
rcParams.update(rcParams)

path_AtlasData = '/Users/akhambhati/Remotes/CORE.MRI_Atlases'
path_CoreData = '/Users/akhambhati/Remotes/CORE.RAM_Stim'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.RAM_Stim'
path_InpData = path_PeriphData + '/e01-FuncNetw'
path_InpData_Baseline = path_PeriphData + '/e01b-FuncNetw'
path_ExpData = path_PeriphData + '/e02-GlobalTopo'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

# Gather all data sets

## Baseline data

In [ ]:
base_list = glob.glob('{}/Adjacency.*.npz'.format(path_InpData_Baseline))
all_subj_id = np.unique([pth.split('/')[-1].split('.')[1] for pth in base_list])

base_dict = {'Subject_ID': [],
             'Base_ID': [],
             'Adj_Path': []}
for subj_id in all_subj_id:
    print(subj_id)
    
    ### Iterate over all baseline events for the subject
    for pth in glob.glob('{}/Adjacency.{}.*.npz'.format(path_InpData_Baseline, subj_id)):    
        full_file = pth.split('/')[-1]    
        subj_id = full_file.split('.')[1]
        base_id = int(full_file.split('.')[2].split('_')[-1])
        
        ### Populate the dictionary
        base_dict['Subject_ID'].append(subj_id)
        base_dict['Base_ID'].append(base_id)    
        base_dict['Adj_Path'].append(pth)

pkl.dump(base_dict, open('{}/base_dict.pkl'.format(path_ExpData), 'w'))  
print('There are {} processed baseline events to analyze.'.format(len(base_dict['Subject_ID'])))

## Stim data

In [ ]:
trial_list = glob.glob('{}/Adjacency.*.npz'.format(path_InpData))
all_subj_id = np.unique([pth.split('/')[-1].split('.')[1] for pth in trial_list])


trial_dict = {'Subject_ID': [],
              'Experiment_ID': [],
              'Trial_ID': [],
              'Event_ID': [],
              'Stim_Freq': [],
              'Stim_Amp': [],
              'Stim_Anode': [],
              'Stim_Cathode': [],           
              'Stim_Anode_Loc': [],
              'Stim_Cathode_Loc': [],                         
              'Dist_To_Anode': [],
              'Dist_To_Cathode': [],
              'Adj_Path': []}


for subj_id in all_subj_id:
    print(subj_id)
    
    ### Get channel map for subject
    chan_file = glob.glob('{}/Exp_Info/Channel_Info/{}.mat'.format(path_CoreData, subj_id))
    if len(chan_file) != 1:
        continue
    df_chan = io.loadmat(chan_file[0])
    if df_chan['good_channels_jack'].shape[1] != df_chan['good_channels_ind'].shape[1]:
        continue
        
    ### Compute Inter-Electrode Distances        
    n_node = df_chan['good_channels_jack'].shape[1]
    dist_matr = np.zeros((n_node, n_node))
    triu_ix, triu_iy = np.triu_indices(n_node, k=1)
    for ix, iy in zip(triu_ix, triu_iy):
        ix_chan_jack = df_chan['good_channels_jack'][0, ix]
        iy_chan_jack = df_chan['good_channels_jack'][0, iy]
        try:
            ix_chan_loc = df_chan['xyzcoords'][df_chan['xyzcoords'][:, 0] == ix_chan_jack, 1:][0, :]
            iy_chan_loc = df_chan['xyzcoords'][df_chan['xyzcoords'][:, 0] == iy_chan_jack, 1:][0, :]
            dist_matr[ix, iy] = np.sqrt(np.sum((ix_chan_loc-iy_chan_loc)**2))
        except:
            dist_matr[ix, iy] = np.nan
    dist_matr += dist_matr.T

    ### Get Trial LUT for Subject
    trial_lut_file = glob.glob('{}/Exp_Info/LUT_Trial_Events/{}_trial_lut.mat'.format(path_CoreData, subj_id))
    if len(trial_lut_file) != 1:
        continue
    df_lut = h5py.File(trial_lut_file[0], 'r')
    
    ### Get event table for subject
    event_file = glob.glob('{}/Exp_Info/PS_Events/{}_events.mat'.format(path_CoreData, subj_id))
    if len(event_file) != 1:
        continue
    df_event = io.loadmat(event_file[0])

    ### Iterate over all trials for the subject
    for pth in glob.glob('{}/Adjacency.{}.*.npz'.format(path_InpData, subj_id)):    
        full_file = pth.split('/')[-1]    
        subj_id = full_file.split('.')[1]
        trial_id = int(full_file.split('.')[2].split('_')[-1])

        ### Convert the Trial ID to Event ID
        event_id = df_lut['trial_lut'][1, :][df_lut['trial_lut'][0, :] == trial_id]
        if len(event_id) != 1:
            continue
        event_id = int(event_id[0] - 1)

        ### Get Trial Parameters
        sel_event = df_event['events'][0, event_id]
        stim_exp = sel_event['experiment'][0]        
        stim_amp = sel_event['amplitude'][0, 0]
        stim_freq = sel_event['pulse_frequency'][0, 0]

        stim_anode_jack = df_event['events'][0, event_id]['stimAnode'][0, 0]
        stim_cathode_jack = df_event['events'][0, event_id]['stimCathode'][0, 0]

        stim_anode_ix = np.flatnonzero(df_chan['good_channels_jack'][0,:] == stim_anode_jack)[0]
        stim_cathode_ix = np.flatnonzero(df_chan['good_channels_jack'][0,:] == stim_cathode_jack)[0]
        
        stim_anode_loc = df_chan['xyzcoords'][df_chan['xyzcoords'][:, 0] == stim_anode_jack, 1:][0, :]
        stim_cathode_loc = df_chan['xyzcoords'][df_chan['xyzcoords'][:, 0] == stim_cathode_jack, 1:][0, :]        

        ### Get euclidean distances to stim node
        dist_to_anode = dist_matr[stim_anode_ix, :]
        dist_to_anode = np.delete(dist_to_anode, (stim_anode_ix, stim_cathode_ix))
        dist_to_cathode = dist_matr[stim_cathode_ix, :]
        dist_to_cathode = np.delete(dist_to_cathode, (stim_anode_ix, stim_cathode_ix))
        
        ### Populate the dictionary
        trial_dict['Subject_ID'].append(subj_id)
        trial_dict['Experiment_ID'].append(stim_exp)        
        trial_dict['Trial_ID'].append(trial_id)    
        trial_dict['Event_ID'].append(event_id)
        trial_dict['Stim_Freq'].append(stim_freq)
        trial_dict['Stim_Amp'].append(stim_amp)    
        trial_dict['Stim_Anode'].append(stim_anode_ix)
        trial_dict['Stim_Cathode'].append(stim_cathode_ix)
        trial_dict['Stim_Anode_Loc'].append(stim_anode_loc)
        trial_dict['Stim_Cathode_Loc'].append(stim_cathode_loc)        
        trial_dict['Dist_To_Anode'].append(dist_to_anode)
        trial_dict['Dist_To_Cathode'].append(dist_to_cathode)
        trial_dict['Adj_Path'].append(pth)
    df_lut.close()

pkl.dump(trial_dict, open('{}/stim_trial_dict.pkl'.format(path_ExpData), 'w'))  
print('There are {} processed trials to analyze.'.format(len(trial_dict['Subject_ID'])))

## Load Cached Data

In [ ]:
base_dict = pkl.load(open('{}/base_dict.pkl'.format(path_ExpData), 'r'))
df_base = pd.DataFrame(base_dict, columns=base_dict.keys())

trial_dict = pkl.load(open('{}/stim_trial_dict.pkl'.format(path_ExpData), 'r'))
df_trial = pd.DataFrame(trial_dict, columns=trial_dict.keys())

# Select a subset of the trials
overlap_subj_ids = np.intersect1d(np.unique(df_trial['Subject_ID']),
                                  np.unique(df_base['Subject_ID']))

# Visualize stim locations

In [ ]:
cmap = plt.cm.get_cmap('RdBu')
norm = plt.Normalize(vmin=-1.5, vmax=1.5)

df_base_topo = pd.read_pickle('{}/baseline_topology.pkl'.format(path_ExpData))

adj_freqs = ['ZNodeStr_AlphaTheta', 'ZNodeStr_Beta',
             'ZNodeStr_LowGamma', 'ZNodeStr_HighGamma']
for afreq in adj_freqs:
    
    # Grab Z-Score node strength for stim channel and the stim channel location
    stim_loc = []
    stim_zns = []
    for subj_id in overlap_subj_ids:
        zns = np.array(df_base_topo[df_base_topo['Subject_ID'] == subj_id][afreq])[0]
        
        sel_trial = df_trial[df_trial['Subject_ID'] == subj_id]
        stim_ix = np.unique(sel_trial['Stim_Anode'])
        for ix in stim_ix:
            stim_loc.append(list(np.array(sel_trial[sel_trial['Stim_Anode'] == ix]['Stim_Anode_Loc'])[0]))
            stim_zns.append(cmap(norm(zns[ix])))
    stim_loc = np.array(stim_loc)

    # Plot the stim node strength
    view_angle = {'Axial_LR': [0.0, 0.0],
                  'Axial_RL': [0.0, 180.0],
                  'Sag_PA': [0.0, 90.0],
                  'Sag_AP': [180.0, 90.0]}

    Echobase.Plotting.render_brain_connectivity.draw(vtk_files=['{}/fsl/MNI/rh.pial.vtk'.format(path_AtlasData),
                                                                '{}/fsl/MNI/lh.pial.vtk'.format(path_AtlasData)],
                                                     brain_rgba=(0.5, 0.5, 0.5, 0.05), conn_cmap='inferno',
                                                     node_coords=stim_loc, node_rgba=stim_zns, node_rad=5.0,
                                                     conn_thr=[0.975, 1.0], conn_list=None)
    for ang in view_angle.keys():
        Echobase.Plotting.render_brain_connectivity.mlab.view(azimuth=view_angle[ang][0],
                                                              elevation=view_angle[ang][1])
        Echobase.Plotting.render_brain_connectivity.mlab.savefig('./e02-Figures/StimLocs-{}-{}.png'.format(afreq, ang))
    Echobase.Plotting.render_brain_connectivity.mlab.close(all=True)

# Measure Global Topology

## Baseline Topology

In [ ]:
data_table_1 = {'Subject_ID': [],
                                
                'Corr_Conn_AlphaTheta': [],
                'Corr_Conn_Beta': [],
                'Corr_Conn_LowGamma': [],                
                'Corr_Conn_HighGamma': [],
                
                'ZNodeStr_AlphaTheta': [],
                'ZNodeStr_Beta': [],
                'ZNodeStr_LowGamma': [],               
                'ZNodeStr_HighGamma': []}

for subj_id in overlap_subj_ids:
    sel_subj = df_base[df_base['Subject_ID'] == subj_id]

    cfg_vec_alphatheta = []
    cfg_vec_beta = []
    cfg_vec_lowgamma = []
    cfg_vec_highgamma = []       
    
    ns_alphatheta = []
    ns_beta = []
    ns_lowgamma = []
    ns_highgamma = []
        
    for base_ix, base_data in sel_subj.iterrows():
        # Load Adjacency
        df_adj = np.load(base_data['Adj_Path'])
        adj = df_adj['adj'].item()   
        df_adj.close()

        # Grab adjacency matrices
        adj_alphatheta = adj['No_Stim']['AlphaTheta']
        adj_beta = adj['No_Stim']['Beta']
        adj_lowgamma = adj['No_Stim']['LowGamma']
        adj_highgamma = adj['No_Stim']['HighGamma']  
        
        # Compute node strength
        ns_alphatheta.append(np.mean(adj_alphatheta, axis=0))
        ns_beta.append(np.mean(adj_beta, axis=0))
        ns_lowgamma.append(np.mean(adj_lowgamma, axis=0))
        ns_highgamma.append(np.mean(adj_highgamma, axis=0))        
        
        # Get configuration vector
        cfg_alphatheta = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_alphatheta, axis=0)).reshape(-1)
        cfg_beta = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_beta, axis=0)).reshape(-1)
        cfg_lowgamma = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_lowgamma, axis=0)).reshape(-1)
        cfg_highgamma = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_highgamma, axis=0)).reshape(-1)    
    
        cfg_vec_alphatheta.append(cfg_alphatheta)
        cfg_vec_beta.append(cfg_beta)
        cfg_vec_lowgamma.append(cfg_lowgamma)
        cfg_vec_highgamma.append(cfg_highgamma)      
    
    # Average the node strengths across baseline events
    ns_alphatheta = np.mean(np.array(ns_alphatheta), axis=0)
    ns_beta = np.mean(np.array(ns_beta), axis=0)
    ns_lowgamma = np.mean(np.array(ns_lowgamma), axis=0)
    ns_highgamma = np.mean(np.array(ns_highgamma), axis=0)

    # Compute Z-Scored node strengths
    zns_alphatheta = (ns_alphatheta - np.nanmean(ns_alphatheta)) / np.nanstd(ns_alphatheta)   
    zns_beta = (ns_beta - np.nanmean(ns_beta)) / np.nanstd(ns_beta)   
    zns_lowgamma = (ns_lowgamma - np.nanmean(ns_lowgamma)) / np.nanstd(ns_lowgamma)   
    zns_highgamma = (ns_highgamma - np.nanmean(ns_highgamma)) / np.nanstd(ns_highgamma)
    
    # Compute correlations between configurations of different baseline time points
    cfg_vec_alphatheta = np.array(cfg_vec_alphatheta)
    cfg_vec_beta = np.array(cfg_vec_beta)
    cfg_vec_lowgamma = np.array(cfg_vec_lowgamma)
    cfg_vec_highgamma = np.array(cfg_vec_highgamma)             
        
    corr_c_alphatheta = convert_adj_matr_to_cfg_matr(np.expand_dims(np.corrcoef(cfg_vec_alphatheta), axis=0)).mean()
    corr_c_beta = convert_adj_matr_to_cfg_matr(np.expand_dims(np.corrcoef(cfg_vec_beta), axis=0)).mean()
    corr_c_lowgamma = convert_adj_matr_to_cfg_matr(np.expand_dims(np.corrcoef(cfg_vec_lowgamma), axis=0)).mean()
    corr_c_highgamma = convert_adj_matr_to_cfg_matr(np.expand_dims(np.corrcoef(cfg_vec_highgamma), axis=0)).mean()    
    
    # Add to Data Table 1
    data_table_1['Subject_ID'].append(subj_id)

    data_table_1['Corr_Conn_AlphaTheta'].append(corr_c_alphatheta)
    data_table_1['Corr_Conn_Beta'].append(corr_c_beta)
    data_table_1['Corr_Conn_LowGamma'].append(corr_c_lowgamma)
    data_table_1['Corr_Conn_HighGamma'].append(corr_c_highgamma)

    data_table_1['ZNodeStr_AlphaTheta'].append(zns_alphatheta)
    data_table_1['ZNodeStr_Beta'].append(zns_beta)
    data_table_1['ZNodeStr_LowGamma'].append(zns_lowgamma)
    data_table_1['ZNodeStr_HighGamma'].append(zns_highgamma)    
    
# Save Data Tables for R-stats
df_base_topo = pd.DataFrame(data_table_1, columns=data_table_1.keys())
df_base_topo.to_pickle('{}/baseline_topology.pkl'.format(path_ExpData))
#df_base_topo.to_csv('{}/baseline_topology.csv'.format(path_ExpData))

## Stim Topology

In [ ]:
data_table_1 = {'Subject_ID': [],
                'Trial_ID': [],
                'Stim_Freq': [],
                'Stim_Amp': [],
                'Stim_Anode': [],
                'Stim_Cathode': [],
                
                'Pre_MeanConnStr_AlphaTheta': [],
                'Pre_MeanConnStr_Beta': [],
                'Pre_MeanConnStr_LowGamma': [],
                'Pre_MeanConnStr_HighGamma': [],    
                
                'Post_MeanConnStr_AlphaTheta': [],
                'Post_MeanConnStr_Beta': [],
                'Post_MeanConnStr_LowGamma': [],
                'Post_MeanConnStr_HighGamma': [],
                                
                'Pre_VarConnStr_AlphaTheta': [],
                'Pre_VarConnStr_Beta': [],
                'Pre_VarConnStr_LowGamma': [],
                'Pre_VarConnStr_HighGamma': [],    
                
                'Post_VarConnStr_AlphaTheta': [],
                'Post_VarConnStr_Beta': [],
                'Post_VarConnStr_LowGamma': [],
                'Post_VarConnStr_HighGamma': [],
                
                'Corr_Conn_AlphaTheta': [],
                'Corr_Conn_Beta': [],
                'Corr_Conn_LowGamma': [],                
                'Corr_Conn_HighGamma': [],

                'Delta_MeanConnStr_AlphaTheta': [],
                'Delta_MeanConnStr_Beta': [],
                'Delta_MeanConnStr_LowGamma': [],
                'Delta_MeanConnStr_HighGamma': [],

                'Delta_VarConnStr_AlphaTheta': [],
                'Delta_VarConnStr_Beta': [],
                'Delta_VarConnStr_LowGamma': [],
                'Delta_VarConnStr_HighGamma': [],
                
                'Delta_PosNodeStr_AlphaTheta': [],
                'Delta_PosNodeStr_Beta': [],
                'Delta_PosNodeStr_LowGamma': [],
                'Delta_PosNodeStr_HighGamma': [],

                'Delta_NegNodeStr_AlphaTheta': [],
                'Delta_NegNodeStr_Beta': [],
                'Delta_NegNodeStr_LowGamma': [],
                'Delta_NegNodeStr_HighGamma': [],
               
                'Delta_PosNodeID_AlphaTheta': [],
                'Delta_PosNodeID_Beta': [],
                'Delta_PosNodeID_LowGamma': [],
                'Delta_PosNodeID_HighGamma': [],

                'Delta_NegNodeID_AlphaTheta': [],
                'Delta_NegNodeID_Beta': [],
                'Delta_NegNodeID_LowGamma': [],
                'Delta_NegNodeID_HighGamma': []}

for subj_id in overlap_subj_ids:
    sel_subj = df_trial[df_trial['Subject_ID'] == subj_id]

    for trial_ix, trial_data in sel_subj.iterrows():
        if (trial_ix % 1000) == 0:
            print(trial_ix)

        # Load Adjacency
        df_adj = np.load(trial_data['Adj_Path'])
        adj = df_adj['adj'].item()   
        df_adj.close()

        # Grab adjacency matrices
        adj_pre_alphatheta = adj['Pre_Stim']['AlphaTheta']
        adj_pre_beta = adj['Pre_Stim']['Beta']
        adj_pre_lowgamma = adj['Pre_Stim']['LowGamma']
        adj_pre_highgamma = adj['Pre_Stim']['HighGamma']    
        cfg_pre_alphatheta = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_pre_alphatheta, axis=0)).reshape(-1)
        cfg_pre_beta = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_pre_beta, axis=0)).reshape(-1)
        cfg_pre_lowgamma = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_pre_lowgamma, axis=0)).reshape(-1)
        cfg_pre_highgamma = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_pre_highgamma, axis=0)).reshape(-1)    

        adj_post_alphatheta = adj['Post_Stim']['AlphaTheta']
        adj_post_beta = adj['Post_Stim']['Beta']
        adj_post_lowgamma = adj['Post_Stim']['LowGamma']
        adj_post_highgamma = adj['Post_Stim']['HighGamma'] 
        cfg_post_alphatheta = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_post_alphatheta, axis=0)).reshape(-1)
        cfg_post_beta = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_post_beta, axis=0)).reshape(-1)
        cfg_post_lowgamma = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_post_lowgamma, axis=0)).reshape(-1)
        cfg_post_highgamma = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_post_highgamma, axis=0)).reshape(-1)    

        # Compute pre/post matrices (mean conn strength, var conn strength, corr conn profile)
        pre_mcs_alphatheta = np.mean(cfg_pre_alphatheta)
        pre_mcs_beta = np.mean(cfg_pre_beta)
        pre_mcs_lowgamma = np.mean(cfg_pre_lowgamma)
        pre_mcs_highgamma = np.mean(cfg_pre_highgamma)

        post_mcs_alphatheta = np.mean(cfg_post_alphatheta)
        post_mcs_beta = np.mean(cfg_post_beta)
        post_mcs_lowgamma = np.mean(cfg_post_lowgamma)
        post_mcs_highgamma = np.mean(cfg_post_highgamma)

        pre_vcs_alphatheta = np.var(cfg_pre_alphatheta)
        pre_vcs_beta = np.var(cfg_pre_beta)
        pre_vcs_lowgamma = np.var(cfg_pre_lowgamma)
        pre_vcs_highgamma = np.var(cfg_pre_highgamma)

        post_vcs_alphatheta = np.var(cfg_post_alphatheta)
        post_vcs_beta = np.var(cfg_post_beta)
        post_vcs_lowgamma = np.var(cfg_post_lowgamma)
        post_vcs_highgamma = np.var(cfg_post_highgamma)

        corr_c_alphatheta = stats.pearsonr(cfg_pre_alphatheta, cfg_post_alphatheta)[0]
        corr_c_beta = stats.pearsonr(cfg_pre_beta, cfg_post_beta)[0]
        corr_c_lowgamma = stats.pearsonr(cfg_pre_lowgamma, cfg_post_lowgamma)[0]
        corr_c_highgamma = stats.pearsonr(cfg_pre_highgamma, cfg_post_highgamma)[0]    

        # Compute Delta Matrices (node strength, mean node strength, var node strength)
        adj_delta_alphatheta = adj['Post_Stim']['AlphaTheta'] - adj['Pre_Stim']['AlphaTheta']
        adj_delta_beta = adj['Post_Stim']['Beta'] - adj['Pre_Stim']['Beta']
        adj_delta_lowgamma = adj['Post_Stim']['LowGamma'] - adj['Pre_Stim']['LowGamma']
        adj_delta_highgamma = adj['Post_Stim']['HighGamma'] - adj['Pre_Stim']['HighGamma'] 
        cfg_delta_alphatheta = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_delta_alphatheta, axis=0)).reshape(-1)
        cfg_delta_beta = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_delta_beta, axis=0)).reshape(-1)
        cfg_delta_lowgamma = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_delta_lowgamma, axis=0)).reshape(-1)
        cfg_delta_highgamma = convert_adj_matr_to_cfg_matr(np.expand_dims(adj_delta_highgamma, axis=0)).reshape(-1)            

        delta_mcs_alphatheta = np.mean(cfg_delta_alphatheta)
        delta_mcs_beta = np.mean(cfg_delta_beta)
        delta_mcs_lowgamma = np.mean(cfg_delta_lowgamma)
        delta_mcs_highgamma = np.mean(cfg_delta_highgamma)

        delta_vcs_alphatheta = np.var(cfg_delta_alphatheta)
        delta_vcs_beta = np.var(cfg_delta_beta)
        delta_vcs_lowgamma = np.var(cfg_delta_lowgamma)
        delta_vcs_highgamma = np.var(cfg_delta_highgamma)
        
        # Maximum positive and negative change in node strength        
        delta_ns_alphatheta = np.mean(adj_delta_alphatheta, axis=0)
        delta_ns_beta = np.mean(adj_delta_beta, axis=0)
        delta_ns_lowgamma = np.mean(adj_delta_lowgamma, axis=0)
        delta_ns_highgamma = np.mean(adj_delta_highgamma, axis=0)
    
        delta_pns_alphatheta = np.max(delta_ns_alphatheta)
        delta_pns_beta = np.max(delta_ns_beta)
        delta_pns_lowgamma = np.max(delta_ns_lowgamma)
        delta_pns_highgamma = np.max(delta_ns_highgamma)

        delta_nns_alphatheta = np.min(delta_ns_alphatheta)
        delta_nns_beta = np.min(delta_ns_beta)
        delta_nns_lowgamma = np.min(delta_ns_lowgamma)
        delta_nns_highgamma = np.min(delta_ns_highgamma)

        delta_pni_alphatheta = np.argmax(delta_ns_alphatheta)
        delta_pni_beta = np.argmax(delta_ns_beta)
        delta_pni_lowgamma = np.argmax(delta_ns_lowgamma)
        delta_pni_highgamma = np.argmax(delta_ns_highgamma)

        delta_nni_alphatheta = np.argmin(delta_ns_alphatheta)
        delta_nni_beta = np.argmin(delta_ns_beta)
        delta_nni_lowgamma = np.argmin(delta_ns_lowgamma)
        delta_nni_highgamma = np.argmin(delta_ns_highgamma)
        
        # Add to Data Table 1
        data_table_1['Subject_ID'].append(trial_data['Subject_ID'])
        data_table_1['Trial_ID'].append(trial_data['Trial_ID'])    
        data_table_1['Stim_Freq'].append(trial_data['Stim_Freq'])
        data_table_1['Stim_Amp'].append(trial_data['Stim_Amp'])
        data_table_1['Stim_Anode'].append(trial_data['Stim_Anode'])
        data_table_1['Stim_Cathode'].append(trial_data['Stim_Cathode'])

        data_table_1['Pre_MeanConnStr_AlphaTheta'].append(pre_mcs_alphatheta)
        data_table_1['Pre_MeanConnStr_Beta'].append(pre_mcs_beta)
        data_table_1['Pre_MeanConnStr_LowGamma'].append(pre_mcs_lowgamma)
        data_table_1['Pre_MeanConnStr_HighGamma'].append(pre_mcs_highgamma)

        data_table_1['Post_MeanConnStr_AlphaTheta'].append(post_mcs_alphatheta)
        data_table_1['Post_MeanConnStr_Beta'].append(post_mcs_beta)
        data_table_1['Post_MeanConnStr_LowGamma'].append(post_mcs_lowgamma)
        data_table_1['Post_MeanConnStr_HighGamma'].append(post_mcs_highgamma)

        data_table_1['Pre_VarConnStr_AlphaTheta'].append(pre_vcs_alphatheta)
        data_table_1['Pre_VarConnStr_Beta'].append(pre_vcs_beta)
        data_table_1['Pre_VarConnStr_LowGamma'].append(pre_vcs_lowgamma)
        data_table_1['Pre_VarConnStr_HighGamma'].append(pre_vcs_highgamma)

        data_table_1['Post_VarConnStr_AlphaTheta'].append(post_vcs_alphatheta)
        data_table_1['Post_VarConnStr_Beta'].append(post_vcs_beta)
        data_table_1['Post_VarConnStr_LowGamma'].append(post_vcs_lowgamma)
        data_table_1['Post_VarConnStr_HighGamma'].append(post_vcs_highgamma)

        data_table_1['Corr_Conn_AlphaTheta'].append(corr_c_alphatheta)
        data_table_1['Corr_Conn_Beta'].append(corr_c_beta)
        data_table_1['Corr_Conn_LowGamma'].append(corr_c_lowgamma)
        data_table_1['Corr_Conn_HighGamma'].append(corr_c_highgamma)

        data_table_1['Delta_MeanConnStr_AlphaTheta'].append(delta_mcs_alphatheta)
        data_table_1['Delta_MeanConnStr_Beta'].append(delta_mcs_beta)
        data_table_1['Delta_MeanConnStr_LowGamma'].append(delta_mcs_lowgamma)
        data_table_1['Delta_MeanConnStr_HighGamma'].append(delta_mcs_highgamma)

        data_table_1['Delta_VarConnStr_AlphaTheta'].append(delta_vcs_alphatheta)
        data_table_1['Delta_VarConnStr_Beta'].append(delta_vcs_beta)
        data_table_1['Delta_VarConnStr_LowGamma'].append(delta_vcs_lowgamma)
        data_table_1['Delta_VarConnStr_HighGamma'].append(delta_vcs_highgamma)
        
        data_table_1['Delta_PosNodeStr_AlphaTheta'].append(delta_pns_alphatheta)
        data_table_1['Delta_PosNodeStr_Beta'].append(delta_pns_beta)
        data_table_1['Delta_PosNodeStr_LowGamma'].append(delta_pns_lowgamma)
        data_table_1['Delta_PosNodeStr_HighGamma'].append(delta_pns_highgamma)

        data_table_1['Delta_NegNodeStr_AlphaTheta'].append(delta_nns_alphatheta)
        data_table_1['Delta_NegNodeStr_Beta'].append(delta_nns_beta)
        data_table_1['Delta_NegNodeStr_LowGamma'].append(delta_nns_lowgamma)
        data_table_1['Delta_NegNodeStr_HighGamma'].append(delta_nns_highgamma)

        data_table_1['Delta_PosNodeID_AlphaTheta'].append(delta_pni_alphatheta)
        data_table_1['Delta_PosNodeID_Beta'].append(delta_pni_beta)
        data_table_1['Delta_PosNodeID_LowGamma'].append(delta_pni_lowgamma)
        data_table_1['Delta_PosNodeID_HighGamma'].append(delta_pni_highgamma)

        data_table_1['Delta_NegNodeID_AlphaTheta'].append(delta_nni_alphatheta)
        data_table_1['Delta_NegNodeID_Beta'].append(delta_nni_beta)
        data_table_1['Delta_NegNodeID_LowGamma'].append(delta_nni_lowgamma)
        data_table_1['Delta_NegNodeID_HighGamma'].append(delta_nni_highgamma)
        
# Save Data Tables for R-stats
df_nodestr = pd.DataFrame(data_table_1, columns=data_table_1.keys())
df_nodestr.to_csv('{}/average_node_strength.csv'.format(path_ExpData))

## Global Alterations in Topology

### Mean Connection Strength
The average node strength before stimulation predicts the average node strength after. There is a "conservation of connectivity" for Alpha/Theta, Beta, Low Gamma and High Gamma coherence networks. 

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))

%matplotlib inline

plt.figure(figsize=(3,3))

sel_subj = df.groupby([df['Subject_ID']]).mean().unstack()
adj_freqs = [('Pre_MeanConnStr_AlphaTheta', 'Post_MeanConnStr_AlphaTheta'),
             ('Pre_MeanConnStr_Beta', 'Post_MeanConnStr_Beta'),
             ('Pre_MeanConnStr_LowGamma', 'Post_MeanConnStr_LowGamma'),
             ('Pre_MeanConnStr_HighGamma', 'Post_MeanConnStr_HighGamma')]
for ii, afreq in enumerate(adj_freqs):
    ax = plt.subplot(2, 2, ii+1)
    
    # Regression
    slope, yint, rho, pv, stderr = stats.linregress(sel_subj[afreq[0]], sel_subj[afreq[1]])
    ax.plot([0.0, 1.0], slope*np.array([0.0, 1.0])+yint, 'k', alpha=0.5)    
    ax.scatter(sel_subj[afreq[0]], sel_subj[afreq[1]], s=2.5,
               color='r', lw=0)
    
    ax.text(0.5, 0.75, ' slope=%0.3f\n rho=%0.3f\n pv=%0.3f' % (slope, rho, pv), fontsize=3.0)
    
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    if ii in [0, 2]:
        ax.set_ylabel('Post-Stim\nMean Conn Strength')
    if ii in [2, 3]:
        ax.set_xlabel('Pre-Stim\nMean Conn Strength')      
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_title(afreq[0].split('_')[2])

plt.savefig('./e02-Figures/Pre_MeanConnStrength-Post_MeanConnStrength.png')
plt.show()

### Variance Connection Strength
The average node strength before stimulation predicts the average node strength after. There is a "conservation of connectivity" for Alpha/Theta, Beta, Low Gamma and High Gamma coherence networks. 

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))

%matplotlib inline

plt.figure(figsize=(3,3))

sel_subj = df.groupby([df['Subject_ID']]).mean().unstack()
adj_freqs = [('Pre_VarConnStr_AlphaTheta', 'Post_VarConnStr_AlphaTheta'),
             ('Pre_VarConnStr_Beta', 'Post_VarConnStr_Beta'),
             ('Pre_VarConnStr_LowGamma', 'Post_VarConnStr_LowGamma'),
             ('Pre_VarConnStr_HighGamma', 'Post_VarConnStr_HighGamma')]
for ii, afreq in enumerate(adj_freqs):
    ax = plt.subplot(2, 2, ii+1)
    
    # Regression
    slope, yint, rho, pv, stderr = stats.linregress(sel_subj[afreq[0]], sel_subj[afreq[1]])
    ax.plot([0.0, 0.06], slope*np.array([0.0, 0.06])+yint, 'k', alpha=0.5)    
    ax.scatter(sel_subj[afreq[0]], sel_subj[afreq[1]], s=2.5,
               color='r', lw=0)
    
    ax.text(0.01, 0.04, ' slope=%0.3f\n rho=%0.3f\n pv=%0.3f' % (slope, rho, pv), fontsize=3.0)
    
    ax.set_xlim([0, 0.06])
    ax.set_ylim([0, 0.06])
    if ii in [0, 2]:
        ax.set_ylabel('Post-Stim\nVariance Conn Strength')
    if ii in [2, 3]:
        ax.set_xlabel('Pre-Stim\nVariance Conn Strength')      
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_title(afreq[0].split('_')[2])

plt.savefig('./e02-Figures/Pre_VarConnStrength-Post_VarConnStrength.png')
plt.show()

### Topological Similarity

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))
df_base_topo = pd.read_csv('{}/baseline_topology.csv'.format(path_ExpData))

%matplotlib inline

plt.figure(figsize=(3,3))
ax = plt.subplot(111)

sel_subj_stim = df.groupby([df['Subject_ID']]).mean().unstack()
adj_freqs = ['Corr_Conn_AlphaTheta',
             'Corr_Conn_Beta',
             'Corr_Conn_LowGamma',
             'Corr_Conn_HighGamma']
for ii, afreq in enumerate(adj_freqs):
    bp1 = ax.boxplot([sel_subj_stim[afreq]], positions=[ii], patch_artist=True)
    Echobase.Plotting.fig_format.set_box_color(bp1, 'k', [[0.00, 0.37, 1.00]])
    
    bp2 = ax.boxplot([df_base_topo[afreq]], positions=[ii+0.2], patch_artist=True)
    Echobase.Plotting.fig_format.set_box_color(bp2, 'k', [[0.25, 0.25, 0.25]])    
    print(stats.ttest_rel(sel_subj_stim[afreq], df_base_topo[afreq]))

ax.set_xlim([-0.5, 4])
ax.set_xticks(np.arange(len(adj_freqs))+0.1)
ax.set_xticklabels([afreq.split('_')[-1] for afreq in adj_freqs])
ax.xaxis.set_ticks_position('bottom')

ax.set_ylim([0.0, 1.0])
ax.set_ylabel('Topological Similarity')
ax.yaxis.set_ticks_position('left')

plt.savefig('./e02-Figures/Topological_Similarity-Pre_Post.png')
plt.show()

## Effect of Stimulation Frequency

### Strengthened Hubs

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))
n_null = 10000

%matplotlib inline

plt.figure(figsize=(2,2))
ax = plt.subplot(111)

stim_freqs = np.unique(df['Stim_Freq'])
sel_subj = df.groupby([df['Subject_ID'], df['Stim_Freq']]).mean().unstack()
adj_freqs = ['Delta_PosNodeStr_AlphaTheta',
             'Delta_PosNodeStr_Beta',
             'Delta_PosNodeStr_LowGamma',
             'Delta_PosNodeStr_HighGamma']
for afreq in adj_freqs:
    sel_stim = sel_subj[afreq]                
    
    # Spearman Correlation
    rho, pv = stats.pearsonr(np.log10(stim_freqs), sel_stim.mean())
    null_rho = []
    for n_i in xrange(n_null):
        sel_stim_perm = np.random.permutation(np.array(sel_stim).reshape(-1)).reshape(len(sel_stim), -1)
        null_rho.append(stats.pearsonr(np.log10(stim_freqs),
                                       np.nanmean(sel_stim_perm, axis=0))[0])
    print('%s:  -- Pearson: (rho=%0.4f, pv=%0.4f)' % (afreq, rho, np.mean(np.array(null_rho) > rho)))
        
    ax.errorbar(stim_freqs, np.mean(sel_stim), yerr=np.array(sel_stim.std() / np.sqrt(len(sel_stim))))

ax.legend(['Alpha-Theta', 'Beta', 'Low Gamma', 'High Gamma'], loc=0, fontsize=3.0)

ax.set_xlabel('Stimulation Frequency (Hz)')
ax.set_xlim([10**1, 10**2.5])
ax.set_xscale('log')
ax.xaxis.set_ticks_position('bottom')

ax.set_ylabel('Change in Strength of Strongest Nodes')
ax.set_ylim([0.10, 0.20])
ax.yaxis.set_ticks_position('left')

plt.savefig('./e02-Figures/Stim_Freq-Delta_PosNodeStrength.png')
plt.show()

### Weakened Hubs

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))
n_null = 10000

%matplotlib inline

plt.figure(figsize=(3,3))
ax = plt.subplot(111)

stim_freqs = np.unique(df['Stim_Freq'])
sel_subj = df.groupby([df['Subject_ID'], df['Stim_Freq']]).mean().unstack()
adj_freqs = ['Delta_NegNodeStr_AlphaTheta',
             'Delta_NegNodeStr_Beta',
             'Delta_NegNodeStr_LowGamma',
             'Delta_NegNodeStr_HighGamma']
for afreq in adj_freqs:
    sel_stim = sel_subj[afreq]                
    
    # Spearman Correlation
    rho, pv = stats.pearsonr(np.log10(stim_freqs), sel_stim.mean())
    null_rho = []
    for n_i in xrange(n_null):
        sel_stim_perm = np.random.permutation(np.array(sel_stim).reshape(-1)).reshape(len(sel_stim), -1)
        null_rho.append(stats.pearsonr(np.log10(stim_freqs),
                                       np.nanmean(sel_stim_perm, axis=0))[0])
    print('%s:  -- Pearson: (rho=%0.4f, pv=%0.4f)' % (afreq, rho, np.mean(np.array(null_rho) < rho)))
        
    ax.errorbar(stim_freqs, np.mean(sel_stim), yerr=np.array(sel_stim.std() / np.sqrt(len(sel_stim))))

ax.legend(['Alpha-Theta', 'Beta', 'Low Gamma', 'High Gamma'], loc=0)

ax.set_xlabel('Stimulation Frequency (Hz)')
ax.set_xlim([10**1, 10**2.5])
ax.set_xscale('log')
ax.xaxis.set_ticks_position('bottom')

ax.set_ylabel('Change in Strength of Strongest Nodes')
ax.set_ylim([-0.18, 0.0])
ax.yaxis.set_ticks_position('left')

plt.savefig('./e02-Figures/Stim_Freq-Delta_PosNodeStrength.png')
plt.show()

plt.show()

## Effect of Stimulation Amplitude

### Strengthened Hubs

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))
n_null = 10000

%matplotlib inline

plt.figure()
ax = plt.subplot(111)

stim_amps = np.unique(df['Stim_Amp'])
sel_subj = df.groupby([df['Subject_ID'], df['Stim_Amp']]).mean().unstack()
adj_freqs = ['Delta_PosNodeStr_AlphaTheta',
             'Delta_PosNodeStr_Beta',
             'Delta_PosNodeStr_LowGamma',
             'Delta_PosNodeStr_HighGamma']
for afreq in adj_freqs:
    sel_stim = sel_subj[afreq]                
    
    # Spearman Correlation
    rho, pv = stats.pearsonr(stim_amps, sel_stim.mean())
    null_rho = []
    for n_i in xrange(n_null):
        sel_stim_perm = np.random.permutation(np.array(sel_stim).reshape(-1)).reshape(len(sel_stim), -1)
        null_rho.append(stats.pearsonr(stim_amps,
                                       np.nanmean(sel_stim_perm, axis=0))[0])
    print('%s:  -- Pearson: (rho=%0.4f, pv=%0.4f)' % (afreq, rho, np.mean(np.array(null_rho) < rho)))
        
    ax.errorbar(stim_amps, np.mean(sel_stim), yerr=np.array(sel_stim.std() / np.sqrt(len(sel_stim))))

ax.legend(['Alpha-Theta', 'Beta', 'Low Gamma', 'High Gamma'], loc=0)

ax.set_xlabel('Stimulation Frequency (Hz)')
ax.set_xlim([0.0, 3.5])
ax.xaxis.set_ticks_position('bottom')

ax.set_ylabel('Change in Strength of Strongest Nodes')
ax.yaxis.set_ticks_position('left')

plt.savefig('./e02-Figures/Stim_Freq-Delta_PosNodeStrength.png')
plt.show()

### Weakened Hubs

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))
n_null = 10000

%matplotlib inline

plt.figure()
ax = plt.subplot(111)

stim_amps = np.unique(df['Stim_Amp'])
sel_subj = df.groupby([df['Subject_ID'], df['Stim_Amp']]).mean().unstack()
adj_freqs = ['Delta_NegNodeStr_AlphaTheta',
             'Delta_NegNodeStr_Beta',
             'Delta_NegNodeStr_LowGamma',
             'Delta_NegNodeStr_HighGamma']
for afreq in adj_freqs:
    sel_stim = sel_subj[afreq]                
    
    # Spearman Correlation
    rho, pv = stats.pearsonr(stim_amps, sel_stim.mean())
    null_rho = []
    for n_i in xrange(n_null):
        sel_stim_perm = np.random.permutation(np.array(sel_stim).reshape(-1)).reshape(len(sel_stim), -1)
        null_rho.append(stats.pearsonr(stim_amps,
                                       np.nanmean(sel_stim_perm, axis=0))[0])
    print('%s:  -- Pearson: (rho=%0.4f, pv=%0.4f)' % (afreq, rho, np.mean(np.array(null_rho) < rho)))
        
    ax.errorbar(stim_amps, np.mean(sel_stim), yerr=np.array(sel_stim.std() / np.sqrt(len(sel_stim))))

ax.legend(['Alpha-Theta', 'Beta', 'Low Gamma', 'High Gamma'], loc=0)

ax.set_xlabel('Stimulation Frequency (Hz)')
ax.set_xlim([0.0, 3.5])
ax.xaxis.set_ticks_position('bottom')

ax.set_ylabel('Change in Strength of Strongest Nodes')
ax.yaxis.set_ticks_position('left')

plt.savefig('./e02-Figures/Stim_Freq-Delta_PosNodeStrength.png')
plt.show()

plt.show()

## Stimulation Location

### Effect on Mean Change in Connection Strength

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))
df_base_topo = pd.read_pickle('{}/baseline_topology.pkl'.format(path_ExpData))

%matplotlib inline

plt.figure(figsize=(3,3))
ax = plt.subplot(111)
    
adj_freqs = [('Delta_MeanConnStr_AlphaTheta', 'ZNodeStr_AlphaTheta'),
             ('Delta_MeanConnStr_Beta', 'ZNodeStr_Beta'),
             ('Delta_MeanConnStr_LowGamma', 'ZNodeStr_LowGamma'),
             ('Delta_MeanConnStr_HighGamma', 'ZNodeStr_HighGamma')]
for ii, afreq in enumerate(adj_freqs):
    ax = plt.subplot(2, 2, ii+1)
    
    subj_stim_res = []
    subj_stim_zns = []
    for subj_id in overlap_subj_ids:
        zns = np.array(df_base_topo[df_base_topo['Subject_ID'] == subj_id][afreq[1]])[0]
        sel_subj = df[df['Subject_ID'] == subj_id]
        stim_ix = np.unique(sel_subj['Stim_Anode'])
        for ix in stim_ix:
            sel_stim = sel_subj[sel_subj['Stim_Anode'] == ix]
            subj_stim_res.append(sel_stim[afreq[0]].mean())
            subj_stim_zns.append(zns[ix])
            
    # Regression
    slope, yint, rho, pv, stderr = stats.linregress(subj_stim_zns, subj_stim_res)
    ax.plot([-3, 3], slope*np.array([-3, 3])+yint, 'k', alpha=0.5)    
    ax.scatter(subj_stim_zns, subj_stim_res, s=3.0,
               color='r', lw=0)
    
    rho, pv = stats.spearmanr(subj_stim_zns, subj_stim_res)
    ax.text(1.0, 0.05, 'rho=%0.3f\npv=%0.3f' % (rho, pv), fontsize=4.0)
    
    ax.set_xlim([-4, 4])
    ax.set_ylim([-0.15, 0.15])
    if ii in [0, 2]:
        ax.set_ylabel('Mean Evoked\nConnection Strength')
    if ii in [2, 3]:
        ax.set_xlabel('Stim Node Strength')
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_title(afreq[0].split('_')[-1])

plt.savefig('./e02-Figures/ZNodeStr-Delta_MeanConnStrength.png')
plt.show()

### Effect on Variance of Change in Connection Strength

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))
df_base_topo = pd.read_pickle('{}/baseline_topology.pkl'.format(path_ExpData))

%matplotlib inline

plt.figure(figsize=(3,3))
ax = plt.subplot(111)
    
adj_freqs = [('Delta_VarConnStr_AlphaTheta', 'ZNodeStr_AlphaTheta'),
             ('Delta_VarConnStr_Beta', 'ZNodeStr_Beta'),
             ('Delta_VarConnStr_LowGamma', 'ZNodeStr_LowGamma'),
             ('Delta_VarConnStr_HighGamma', 'ZNodeStr_HighGamma')]
for ii, afreq in enumerate(adj_freqs):
    ax = plt.subplot(2, 2, ii+1)
    
    subj_stim_res = []
    subj_stim_zns = []
    for subj_id in overlap_subj_ids:
        zns = np.array(df_base_topo[df_base_topo['Subject_ID'] == subj_id][afreq[1]])[0]
        sel_subj = df[df['Subject_ID'] == subj_id]
        stim_ix = np.unique(sel_subj['Stim_Anode'])
        for ix in stim_ix:
            sel_stim = sel_subj[sel_subj['Stim_Anode'] == ix]
            subj_stim_res.append(sel_stim[afreq[0]].mean())
            subj_stim_zns.append(zns[ix])
            
    # Regression
    slope, yint, rho, pv, stderr = stats.linregress(subj_stim_zns, subj_stim_res)
    ax.plot([-3, 3], slope*np.array([-3, 3])+yint, 'k', alpha=0.5)    
    ax.scatter(subj_stim_zns, subj_stim_res, s=5,
               color='r', lw=0)

    rho, pv = stats.spearmanr(subj_stim_zns, subj_stim_res)    
    ax.text(1.0, 0.05, 'rho=%0.3f\npv=%0.3f' % (rho, pv), fontsize=5.0)
    
    ax.set_xlim([-4, 4])
    if ii in [0, 2]:
        ax.set_ylabel('Variance Change in\nConnection Strength')
    if ii in [2, 3]:
        ax.set_xlabel('Stim Node Strength')
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_title(afreq[0].split('_')[-1])            

plt.savefig('./e02-Figures/ZNodeStr-Delta_VarConnStrength.png')
plt.show()

### Effect on Strongest of Changes in Connection Strength

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))
df_base_topo = pd.read_pickle('{}/baseline_topology.pkl'.format(path_ExpData))

%matplotlib inline

plt.figure(figsize=(3,3))
ax = plt.subplot(111)
    
adj_freqs = [(('Delta_PosNodeStr_AlphaTheta', 'Delta_NegNodeStr_AlphaTheta'), 'ZNodeStr_AlphaTheta'),
             (('Delta_PosNodeStr_Beta', 'Delta_NegNodeStr_Beta'), 'ZNodeStr_Beta'),
             (('Delta_PosNodeStr_LowGamma', 'Delta_NegNodeStr_LowGamma'), 'ZNodeStr_LowGamma'),
             (('Delta_PosNodeStr_HighGamma', 'Delta_NegNodeStr_HighGamma'), 'ZNodeStr_HighGamma')]
for ii, afreq in enumerate(adj_freqs):
    ax = plt.subplot(2, 2, ii+1)
    
    for aafreq, clr, coord in zip(afreq[0], ('r', 'b'), ((-3.75, -0.05), (2.0, -0.35))):
        subj_stim_res = []
        subj_stim_zns = []
        for subj_id in overlap_subj_ids:
            zns = np.array(df_base_topo[df_base_topo['Subject_ID'] == subj_id][afreq[1]])[0]
            sel_subj = df[df['Subject_ID'] == subj_id]
            stim_ix = np.unique(sel_subj['Stim_Anode'])
            for ix in stim_ix:
                sel_stim = sel_subj[sel_subj['Stim_Anode'] == ix]
                subj_stim_res.append(sel_stim[aafreq].mean())
                subj_stim_zns.append(zns[ix])

        # Regression
        slope, yint, rho, pv, stderr = stats.linregress(subj_stim_zns, subj_stim_res)
        ax.plot([-3, 3], slope*np.array([-3, 3])+yint, color=clr, alpha=0.5, lw=1.0)    
        ax.scatter(subj_stim_zns, subj_stim_res, s=1.0,
                   color=clr, lw=0)

        rho, pv = stats.spearmanr(subj_stim_zns, subj_stim_res)    
        ax.text(coord[0], coord[1], 'rho=%0.3f\npv=%0.3f' % (rho, pv), fontsize=4.0, color=clr)
    
    ax.set_xlim([-4, 4])
    if ii in [0, 2]:
        ax.set_ylabel('Evoked Node Strength')
    if ii in [2, 3]:
        ax.set_xlabel('Stim Node Strength')
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_title(afreq[0][0].split('_')[-1])            
    
plt.savefig('./e02-Figures/ZNodeStr-Delta_TopNodeStrength.png')
plt.show()

In [ ]:
df = pd.read_csv('{}/average_node_strength.csv'.format(path_ExpData))
df_base_topo = pd.read_pickle('{}/baseline_topology.pkl'.format(path_ExpData))

%matplotlib inline

plt.figure()
ax = plt.subplot(111)
    
adj_freqs = [('Corr_Conn_AlphaTheta', 'ZNodeStr_AlphaTheta'),
             ('Corr_Conn_Beta', 'ZNodeStr_Beta'),
             ('Corr_Conn_LowGamma', 'ZNodeStr_LowGamma'),
             ('Corr_Conn_HighGamma', 'ZNodeStr_HighGamma')]
for ii, afreq in enumerate(adj_freqs):
    ax = plt.subplot(2, 2, ii+1)
    
    subj_stim_res = []
    subj_stim_zns = []
    for subj_id in overlap_subj_ids:
        zns = np.array(df_base_topo[df_base_topo['Subject_ID'] == subj_id][afreq[1]])[0]
        sel_subj = df[df['Subject_ID'] == subj_id]
        stim_ix = np.unique(sel_subj['Stim_Anode'])
        for ix in stim_ix:
            sel_stim = sel_subj[sel_subj['Stim_Anode'] == ix]
            subj_stim_res.append(sel_stim[afreq[0]].mean())
            subj_stim_zns.append(zns[ix])
            
    # Regression
    slope, yint, rho, pv, stderr = stats.linregress(subj_stim_zns, subj_stim_res)
    ax.plot([-3, 3], slope*np.array([-3, 3])+yint, 'k', alpha=0.5)    
    ax.scatter(subj_stim_zns, subj_stim_res, s=5,
               color='r', lw=0)
    
    rho, pv = stats.spearmanr(subj_stim_zns, subj_stim_res)
    ax.text(1.0, 0.05, 'rho=%0.3f\npv=%0.3f' % (rho, pv), fontsize=5.0)
    
    ax.set_xlim([-4, 4])
    #ax.set_ylim([-0.15, 0.15])
    if ii in [0, 2]:
        ax.set_ylabel('Mean Change in\nConnection Strength')
    if ii in [2, 3]:
        ax.set_xlabel('Baseline Stim Node Strength (z-score)')
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_title(afreq[0].split('_')[-1])            
plt.show()